## Notebook 2: Apprentissage automatique, Regression: Polution CO2

#### Importation des librairies

In [4]:
import pandas as pd
import numpy as np
#-------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import *
#-------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer

#--------------------------------------------------------
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

#--------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
#------------------------------------------------
from xgboost import XGBRegressor

#-----------------------------------------------------------
from sklearn.model_selection import GridSearchCV

## Checklist

Depuis 2001, **l’ADEME** acquiert tous les ans ces données auprès de **l’Union Technique de l’Automobile du motocycle et du Cycle UTAC** (en charge de l’homologation des véhicules avant leur mise en vente) en accord avec le ministère du développement durable.
Pour chaque véhicule les données d’origine (transmises par l’Utac) sont les suivantes :

* **Les consommations de carburant**

* **Les émissions de dioxyde de carbone (CO2)**

* **Les émissions des polluants de l’air** (réglementés dans le cadre de la norme Euro)

* **L’ensemble des caractéristiques techniques des véhicules** (gammes, marques, modèles, n° de CNIT, type d’énergie ...)



# L'inventaire des varaibles pertinentes:

Les données comprenent des variables pertinentes suivantes:

* **lib_mrq_utac**: La marque, il y'a 12.

* **lib_mod**: Le modèle commerciale, il y'a 20.

* **cod_cbr**: Le type de carburant, il y a 5.

* **hybride**: Information permettant d’identifier les véhicules hybrides (O/N)

* **puiss_max** : Puissance maximale

* **typ_boite_nb_rapp**: Type boite de vitesse et le nombre de rapport.

* **conso_urb**: Consommation urbaine de carburant (en l/100km),

* **conso_exurb**: consommation extra urbaine de carburant (en l/100km),

* **conso_mixte**: Consommation mixte de carburant (en l/100km),

* **co2**: Emission de CO2 (en g/km),

* **masse_ordma_min**: Masse en ordre de marche mini

* **masse_ordma_max**: Masse en ordre de marche max
 
* **Carrosserie**: Carrosserie

* **gamme**: Gamme du véhicule



# Objectif

Notre objectif majeur dans ce notebook 2 est de :

Prédire les emisisions de **CO2** des vehicules en fonction de certaines informations (Variables explicatives)

* En utilisant 4 à 5 modéles différents

* En comparant  les scores
    
* En choissisant le meilleur modèle


# Description des données

Lien vers les données: https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/


### Chargement, lecture, apercu et infos des données

In [7]:
data_model = pd.read_csv("data_model.csv")

### Selectionner les feautures les plus importans

In [8]:
New_Data = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]

In [9]:
Y = New_Data['co2']
X = New_Data.drop(['co2'] ,axis =1)

### Traiter la colonne Carrosserie

In [17]:
New_Data["Carrosserie"] = New_Data["Carrosserie"].str.strip()  # j'assemble des données
New_Data["Carrosserie"] = New_Data["Carrosserie"].str.lower() # je les convertis
New_Data["Carrosserie"] = New_Data["Carrosserie"].replace({"suv": "VUS", "berline": "voiture"}) #les placer dans ces labels

print(New_Data["Carrosserie"].head())
carrosserie_counts= New_Data["Carrosserie"].value_counts()
print(carrosserie_counts)

0    voiture
1    voiture
2    voiture
3    voiture
4    voiture
Name: Carrosserie, dtype: object
minibus                46107
voiture                 4411
break                   1226
ts terrains/chemins     1080
coupe                    875
cabriolet                552
monospace compact        319
combispace               250
minispace                145
monospace                 59
combispcace                4
Name: Carrosserie, dtype: int64


/tmp/ipykernel_917/2083813035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Data["Carrosserie"] = New_Data["Carrosserie"].str.strip()  # j'assemble des données
/tmp/ipykernel_917/2083813035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Data["Carrosserie"] = New_Data["Carrosserie"].str.lower() # je les convertis
/tmp/ipykernel_917/2083813035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

### Prediction de CO2
    
Pour chacun de nos modèles:
   * **DummyRegressor**,
   * **LinearRegression**, 
   * **SGDRegressor**, 
   * **RandomForestRegressor**,
   * **GradientBoostingRegressor**,
   * **XGBRegressor**, 
    
Nous allons predire le niveau d'émission de **Co2** puis établir une comparaison entre les performances de chaque modèle.

    Appliquer un GridSearch pour optimiser les hyperparamètres de chaque modèle
    Paramètrer dans GridSearch les scores qui sont adaptés à la régression (R2, MAE et RMSE)
    Identifier le meilleur modèle

In [26]:

# Filtrer les colonnes de type int et float uniquement
numeric_columns = New_Data.select_dtypes(include=['int', 'float']).columns
X = New_Data[numeric_columns]

# Diviser les données en variables dépendantes (Y) et indépendantes (X)
Y = X['co2']
X = X.drop(['co2'], axis=1)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Créer une liste des modèles que vous souhaitez évaluer
models = [
    ('Dummy', DummyRegressor(), {}),
    ('LinearRegression', LinearRegression(), {}),
    ('SGDRegressor', SGDRegressor(), {}),
    ('RandomForestRegressor', RandomForestRegressor(), {}),
    ('GradientBoostingRegressor', GradientBoostingRegressor(), {}),
    ('XGBRegressor', XGBRegressor(), {})
]

# Définir les scores que vous souhaitez utiliser pour l'optimisation (R2, MAE et RMSE)
scoring = {
    'r2': make_scorer(r2_score),
    'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False)
}

# Effectuer la recherche par grille pour chaque modèle
results = {}
for name, model, param_grid in models:
    print(f"Training {name}...")
    grid_search = GridSearchCV(model, param_grid, scoring=scoring, refit='r2', cv=5, verbose=2)
    grid_search.fit(X_train, y_train)
    results[name] = grid_search

# Afficher les résultats
for name, result in results.items():
    print(f"Results for {name}:")
    print("Best R2 score:", result.best_score_)
    print("Best hyperparameters:", result.best_params_)
    print("MAE score on test data:", mean_absolute_error(y_test, result.predict(X_test)))
    print("RMSE score on test data:", mean_squared_error(y_test, result.predict(X_test), squared=False))
    print("===")

Training Dummy...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
Training LinearRegression...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.1s
Training SGDRegressor...


In [31]:
import streamlit as st
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Charger et prétraiter les données
data_model = pd.read_csv("data_model.csv")
New_Data = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]
numeric_columns = ['masse_ordma_min', 'masse_ordma_max']
X = New_Data[numeric_columns]
Y = New_Data['co2']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Définir les modèles et les hyperparamètres à rechercher
models = [
    ('DummyRegressor', DummyRegressor(), {}),
    ('LinearRegression', LinearRegression(), {}),
    ('SGDRegressor', SGDRegressor(), {'max_iter': [100, 500]}),
    ('RandomForestRegressor', RandomForestRegressor(), {'n_estimators': [100, 200, 300]}),
    ('GradientBoostingRegressor', GradientBoostingRegressor(), {'n_estimators': [100, 200, 300]}),
    ('XGBRegressor', XGBRegressor(), {'n_estimators': [100, 200, 300]})
]

scoring = {
    'r2': 'r2',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

# Interface utilisateur Streamlit
st.title('Prédiction de CO2 pour un véhicule')

# Sélection du type de carrosserie
carrosserie_options = sorted(data_model['Carrosserie'].unique())
selected_carrosserie = st.selectbox("Sélectionnez le type de carrosserie :", carrosserie_options)

# Saisie de la masse ordma_min et ordma_max
masse_ordma_min = st.number_input("Entrez la masse ordma_min :", min_value=0)
masse_ordma_max = st.number_input("Entrez la masse ordma_max :", min_value=0)

# Bouton pour lancer la prédiction
if st.button("Prédire CO2"):
    # Filtrer les données en fonction des choix de l'utilisateur
    filtered_data = data_model[
        (data_model['Carrosserie'] == selected_carrosserie) &
        (data_model['masse_ordma_min'] >= masse_ordma_min) &
        (data_model['masse_ordma_max'] <= masse_ordma_max)
    ]

    if not filtered_data.empty:
        st.write("Comparaison des performances des modèles :")

        for name, model, param_grid in models:
            print(f"Training {name}...")
            grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=5, verbose=2)
            grid_search.fit(X_train, y_train)

            # Utiliser le modèle ajusté pour les prédictions
            predictions = grid_search.predict(X_test)
            r2 = r2_score(y_test, predictions)
            mae = mean_absolute_error(y_test, predictions)
            rmse = mean_squared_error(y_test, predictions, squared=False)

            st.write(f"Modèle : {name}")
            st.write(f"Score R2 : {r2:.4f}")
            st.write(f"Mean Absolute Error (MAE) : {mae:.4f}")
            st.write(f"Root Mean Squared Error (RMSE) : {rmse:.4f}")
            st.write("-------------------")
    else:
        st.warning("Aucune donnée disponible pour ces critères.")


### Application Web

Développer une application Streamlit avec les options suivantes :

    1- L'utilisateur doit selectionner le type du carrosserie à partire d'un menu déroulant
    2- L'utilisateur doit saisir 'masse_ordma_min' et 'masse_ordma_max' dans deux champs de saisie différent
    3- Programmer un boutton pour lancer la prédiction de CO2

In [23]:
import streamlit as st
import pandas as pd
import joblib

# Interface utilisateur Streamlit
st.title('Prédiction de CO2 pour un véhicule')

# Sélection du type de carrosserie
carrosserie_options = sorted(data_model['Carrosserie'].unique())
selected_carrosserie = st.selectbox("Sélectionnez le type de carrosserie :", carrosserie_options)

# Saisie de la masse ordma_min et ordma_max
masse_ordma_min = st.number_input("Entrez la masse ordma_min :", min_value=0)
masse_ordma_max = st.number_input("Entrez la masse ordma_max :", min_value=0)

# Bouton pour lancer la prédiction
if st.button("Prédire CO2"):
    # Filtrer les données en fonction des choix de l'utilisateur
    filtered_data = data_model[
        (data_model['Carrosserie'] == selected_carrosserie) &
        (data_model['masse_ordma_min'] >= masse_ordma_min) &
        (data_model['masse_ordma_max'] <= masse_ordma_max)
    ]

    if not filtered_data.empty:
        # Sélectionner les caractéristiques nécessaires pour la prédiction (ajustez en fonction de vos besoins)
        features = filtered_data[['masse_ordma_min', 'masse_ordma_max']]

        # Prédire le CO2
        predictions = model.predict(features)

        # Afficher les prédictions
        st.write("Prédictions de CO2 :")
        st.write(predictions)
    else:
        st.write("Aucune donnée disponible pour ces critères.")


2023-08-22 12:01:12.224 
  command:

    streamlit run /home/yvanoide/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
